# Workshop 14, December 1, 2023

**due by 8:59 pm December 5, 2023**

# Practice Fit with the Final Project Data

- Data for the final project are saved in a h5 file. We will show you how to get this h5 file and how to retrieve the data numpy array
- We will also use this data set to do some binned maximum log likelihood fit exercise
- You should have completed the Part 1 of this problem in WS9.

## Part 1 Loading Data and Plot the Mass Distribution

In [ ]:
import numpy as np
import h5py

In [ ]:
# Downloading this h5 file from the web
import os
os.system("wget https://portal.nersc.gov/project/m3438/physics77/data/datalhc.h5")

In [ ]:
# Open h5 file
h = h5py.File("datalhc.h5",'r')

# Retrieve the data array 
data = h["dataset"][:]

# Check its shape
print(data.shape)

The shape of data array is (1178902, 10), indicating that there are 1,178,902 collision events that contain two photons. The axis 1 has 10 entries. They are

- transverse momentum of photon 1
- pseudo rapidity of photon 1
- azimuthal angle of photon 1
- energy of photon 1
- transverse momentum of photon 2
- pseudo rapidity of photon 2
- azimuthal angle of photon 2
- energy of photon 2
- Event Number, which is an index of the collision event
- Run Number, which is an index of a `run`. At LHC, the detector is often run for an extended period of time, raning from a few minuts to a few hours. Data events collected in the same data taking period are said to be in the same `run`.

In [ ]:
# Here a few functions are defined to get the px, py, pz components of the momentum

def px(pt, phi):
    return pt*np.cos(phi)

def py(pt,phi):
    return pt*np.sin(phi)

def pz(pt, eta):
    return pt*np.sinh(eta)


# Using energy and momentum we can calculate the mass of a particle or a multi-particle system
def mass(E,px,py,pz):
    return np.sqrt(E**2 - (px**2+py**2+pz**2))


In [ ]:
# We will use the functions defined above to calculate 
# the px, py, pz components of the diphoton momentum
px_yy = 
py_yy =
pz_yy = 

# We will also calculate the energy of the diphoton system
# which is the sum of individual photons
E_yy = 

# Finally, we can calculate the diphoton mass
m_yy = mass(E_yy, px_yy,py_yy,pz_yy)

## Plot the $m_{\gamma\gamma}$ distribution
- a tiny bump around 125 GeV is already discernible 

In [ ]:
# How does the diphoton mass distribution look like?

obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160),label='Data 2015-2018')
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')
plt.legend()

## Part 2 Use the maximum log likelihood method to fit a 4th order polynomial to data
- define a 4th order polynomial function, `poly4`
- define the NLL for this `poly4`. 
    - For the sake of simplicity, we calculate the value of the polyminomial at np.linspace(105.5,159.5,55), i.e., the bin centers of the above histogram. The binwidth is 1
- Use scipy.optimize.minimize to perform the fit, in which the coefficients of the terms are determined
- Visualize the fitted Polynomial

In [ ]:
# define a fourth-order polynomial
def poly4(myy,c):
    return 1 + c[0]*myy + c[1]*myy**2 + c[2]*myy**3 + c[3]*myy**4

In [ ]:
def NLLpoly(c, obs):
    # Create a sequence of values at which the expectation will be evaluated
    myy = np.linspace(105.5,159.5,55)
    # The expectaiton. Note that the c[4] factor will take care of the normalization of the polynomial PDF
    exp = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4]
    # return a negative log likelihood
    # again, needs to sum over all 55 bins
    NLLvalue = -1*poisson.logpmf(obs,exp).sum()
    return NLLvalue

### Fit the 4th order polynomial to data
- print out the fit result
- visualize the fitted function and data distribution
**Does the fit make sense?**

In [ ]:
result = minimize(NLLpoly,x0=[-10,100,1,1,1],args=(obs),method='Nelder-Mead')
print(result)

# Pass the fitted values back to the list c
c=result.x

In [ ]:
# Use the fitted coefficients to create the expectation
myy = np.linspace(105.5,159.5,55)
fit = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4]

# Data
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')

## Execute the next cell repeatedly
- the minimization algorithm is pretty rudimentary. It doesn't perform well. 
- the first fit is unlikely to be a good one
- in the next cell, we pass the fit result to c, and use them as the initial value in a new iteration of minimization
- repeatedly execute the next cell and see if the NLL decreases and if the fit continues to improve

**Your final fit plot should look like **

<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig6.png" width=500>

In [ ]:
result = minimize(NLLpoly,x0=c,args=(obs),method='Nelder-Mead')
print('Pay attention to the NLL value {:6.3f}'.format(result.fun))
c=result.x
fit = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4] 
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')

In [ ]:
print("The minimized NLL value with bkg-only PDF is {:5.2f}".format(   ))

## Fit data with a signal plus background PDF
- In this exercise, we will create a different function
$$
f(x=m_{\gamma\gamma}) = n_b \sum_i c_i x^i + n_s \mathrm{Normal}(x,125,1.6)
$$
    - in this setup, the first term is a fourth order polynomial, and its normalized to $n_b$ which is the number of background events. The second term is a normal distribution with a mean of 125 and a standard deviation of 1.6, and its normalization $n_s$ represents the number of signal events. 
    - the shape (mean and std. dev.) of the Gaussian distribution is fixed 
    - the free parameters in the fit include
        - the coefficients of the polynomial
        - the normalizations of the signal and background PDFs ($n_s$ and $n_b$)

In [ ]:
# define the signal pdf

def sigpdf(myy,mean,std):
    return norm.pdf(myy,mean,std)

In [ ]:
# Define negative log likelihood value with the observed data and the signal plus background PDF
def NLLSpluspoly(c, obs):
    myy = np.linspace(105.5,159.5,55)
    exp = poly4(myy, [c[0],c[1],c[2],c[3]])*c[4] + sigpdf(myy,125,1.6)*c[5]
    NLLvalue = -1*poisson.logpmf(obs,exp).sum()
    return NLLvalue

Now we perform an initial fit to the observed data

In [ ]:
# Note that I am using the coefficients determined from the previous fits as the
# initial values here
result = minimize(NLLSpluspoly,x0=np.hstack((c,1000)),args=(obs),method='Nelder-Mead')
c1=result.x
print('Pay attention to the NLL value {:6.3f}'.format(result.fun))
myy = np.linspace(105.5,159.5,55)
fit = poly4(myy, [c1[0],c1[1],c1[2],c1[3]])*c1[4] + sigpdf(myy,125,1.6)*c1[5]
# Data
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')


### Execute the cell below repeatedly to see if fit gets improved?

**Final output**

<img src="https://portal.nersc.gov/project/m3438/physics77/WS09/fig7.png" width=500>

In [ ]:
result = minimize(NLLSpluspoly,x0=c1,args=(obs),method='Nelder-Mead')
c1=result.x
print('Pay attention to the NLL value {:6.3f}'.format(result.fun))
myy = np.linspace(105.5,159.5,55)
fit = poly4(myy, [c1[0],c1[1],c1[2],c1[3]])*c1[4] + sigpdf(myy,125,1.6)*c1[5]
# Data
obs, binedges,others =plt.hist(m_yy,bins=55,range=(105,160), label='data')
plt.plot(myy,fit,'r--', label='Fitted Polynomial')
plt.legend(fontsize=14)
plt.xlabel('$m_{\gamma\gamma}$ [GeV]')
plt.ylabel('Number of entries')


In [ ]:
print('The fitted number of signal events is {:4.2f} '.format(   ))

print("The minimized NLL value with signal-plus-bkg PDF is {:5.2f}".format(   ))

## Exercise 1

Can you make a modification to the functions that perform NLL calculations so that the expectation of each bin is calculated as a numberical integration of the PDF from the lower edge of the bin to the higher edge of the bin?

In [ ]:
# Your code here


## Exercise 2
Can you develop the code to fit the data with a signal plus background pdf where the mean and sigma of the signal pdf are also free?

In [ ]:
def NLLSpluspolyfree(c, obs):

    # complete the cell here
    
    return NLLvalue

In [ ]:
# Your fit code

In [ ]:
# Your fit code

In [ ]:
print('The fitted mean value of the signal is {:4.2f} GeV'.format(  ))
print('The fitted sigma value of the signal is {:4.2f} GeV'.format( ))
print('The fitted number of signal events is {:4.2f} '.format(  ))

Congratulations for completing this workshop!